In [1]:
import os
import numpy as np
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='darkgrid')

In [2]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bananapple","key":"75eb3c0872d58bbdaf9df31824376852"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d kabilan03/dogbreedclassification

 97% 264M/272M [00:04<00:00, 96.8MB/s]
100% 272M/272M [00:04<00:00, 62.0MB/s]


In [5]:
#압축풀기
import zipfile

local_zip = '/content/dogbreedclassification.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/content/dog-breed')
zip_ref.close()

# 경로설정

In [6]:
base_dir='/content/dog-breed/Dog Breed Classification'

train_dir=os.path.join(base_dir,'train')
val_dir=os.path.join(base_dir,'val')
test_dir=os.path.join(base_dir,'test')

In [7]:
train_list=os.listdir(train_dir)
print(len(train_list))

val_list=os.listdir(val_dir)
print(len(val_list))

93
93


In [8]:
sum=0
for i in train_list:
  train_img=os.path.join(train_dir,'{}'.format(i))
  train_img_list=os.listdir(train_img)
  sum+=len(train_img_list)
print(sum)

6391


In [9]:
sum=0
for i in val_list:
  val_img=os.path.join(val_dir,'{}'.format(i))
  val_img_list=os.listdir(val_img)
  sum+=len(val_img_list)
print(sum)

762


# ImageDataGenerator 설정

In [10]:
batch=32
img_width=250
img_height=250

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=30,
                                 shear_range=0.3,
                                 zoom_range=0.3,
                                 horizontal_flip=True
                                 )
val_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  batch_size=batch,
                                                  class_mode='sparse',
                                                  target_size=(img_width,img_height)
                                                  )
val_generator=val_datagen.flow_from_directory(val_dir,
                                              batch_size=batch,
                                              class_mode='sparse',
                                              target_size=(img_width,img_height)
                                              )


Found 6391 images belonging to 93 classes.
Found 762 images belonging to 93 classes.


In [16]:
print(train_generator.class_indices.items())

dict_items([('afghan_hound', 0), ('african_hunting_dog', 1), ('airedale', 2), ('basenji', 3), ('basset', 4), ('beagle', 5), ('bedlington_terrier', 6), ('bernese_mountain_dog', 7), ('black-and-tan_coonhound', 8), ('blenheim_spaniel', 9), ('bloodhound', 10), ('bluetick', 11), ('border_collie', 12), ('border_terrier', 13), ('borzoi', 14), ('boston_bull', 15), ('bouvier_des_flandres', 16), ('brabancon_griffon', 17), ('bull_mastiff', 18), ('cairn', 19), ('cardigan', 20), ('chesapeake_bay_retriever', 21), ('chow', 22), ('clumber', 23), ('cocker_spaniel', 24), ('collie', 25), ('curly-coated_retriever', 26), ('dhole', 27), ('dingo', 28), ('doberman', 29), ('english_foxhound', 30), ('english_setter', 31), ('entlebucher', 32), ('flat-coated_retriever', 33), ('german_shepherd', 34), ('german_short-haired_pointer', 35), ('golden_retriever', 36), ('gordon_setter', 37), ('great_dane', 38), ('great_pyrenees', 39), ('groenendael', 40), ('ibizan_hound', 41), ('irish_setter', 42), ('irish_terrier', 43),

# 모델만들기

In [ ]:
import tensorflow as tf

tf.__version__

from tensorflow.keras import datasets, layers, models
from keras.preprocessing import image
from keras.engine.sequential import Sequential
from tensorflow import keras
from keras import regularizers

In [ ]:
kernel_s=(3,3)

model=models.Sequential()

model.add(layers.Conv2D(32,kernel_s,activation='relu',input_shape=(img_height,img_width,3),
                        kernel_regularizer=regularizers.l2(0.001),padding="VALID"))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64, kernel_s, activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, kernel_s, activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(128,kernel_s,activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,kernel_s,activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(93, activation='softmax'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 124, 124, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_35 (Conv2D)          (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 61, 61, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 59, 59, 64)        36928     
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 29, 29, 64)      

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics='accuracy'
              )

# 학습하기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

earlystopping=EarlyStopping(monitor='val_loss', #val_loss 값이 개선되었을 때
                            patience=5 #5회 epoch 동안 val_loss가 개선되지 않는다면 종료
                            )

history=model.fit(train_generator,
                  epochs=5,
                  validation_data=val_generator,
                  callbacks=[earlystopping]
                  )

Epoch 1/5
200/200 [==============================] - 126s 585ms/step - loss: 4.5357 - accuracy: 0.0141 - val_loss: 4.5305 - val_accuracy: 0.0157
Epoch 2/5
 15/200 [=>............................] - ETA: 1:28 - loss: 4.5302 - accuracy: 0.0208


KeyboardInterrupt



In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epoch_range=range(5)

plt.figrue(figsize=(12,8))
plt.subplot(1,2,1)
plt.plot()